In [1]:
from __future__ import print_function

import rospy

import sys
import copy
import math
import moveit_commander

import moveit_msgs.msg
from moveit_msgs.msg import Constraints, JointConstraint, PositionConstraint, OrientationConstraint, BoundingVolume
from sensor_msgs.msg import JointState, Image
from moveit_msgs.msg import RobotState
import geometry_msgs.msg
from geometry_msgs.msg import Quaternion, Pose, PoseStamped
from std_msgs.msg import String
from moveit_commander.conversions import pose_to_list

from ur_moveit.srv import MoverService, MoverServiceRequest, MoverServiceResponse
from ur_moveit.msg import URMoveitJoints

import jupyros as jr
import os,time,sys

## Subscribe to Robot

In [2]:
rospy.init_node('jupyter_node_sub', anonymous=True)

In [3]:
robot_angles = []

In [4]:
def cb(URMoveitJoints):
    global robot_angles
    # print(URMoveitJoints)
    robot_angles=[URMoveitJoints.joint_00,URMoveitJoints.joint_01,URMoveitJoints.joint_02,URMoveitJoints.joint_03,URMoveitJoints.joint_04,URMoveitJoints.joint_05]
    # print([URMoveitJoints.joint_00,URMoveitJoints.joint_01,URMoveitJoints.joint_02,URMoveitJoints.joint_03,URMoveitJoints.joint_04,URMoveitJoints.joint_05],end='\r')
    # print(URMoveitJoints.joint_00,end='\r')

In [5]:
jr.subscribe('/UR_Q',URMoveitJoints,cb)

In [6]:
robot_angles

[-48.88398361206055,
 -39.4639778137207,
 -25.064029693603516,
 -115.40818786621094,
 89.98777770996094,
 -138.8274688720703]

## Publish to Robot

In [7]:
joint_names = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']

# Between Melodic and Noetic, the return type of plan() changed. moveit_commander has no __version__ variable, so checking the python version as a proxy
if sys.version_info >= (3, 0):
    def planCompat(plan):
        return plan[1]
else:
    def planCompat(plan):
        return plan
        
        
"""
    Given the start angles of the robot, plan a trajectory that ends at the destination pose.
"""
def plan_trajectory(move_group, destination_pose, start_joint_angles): 
    current_joint_state = JointState()
    current_joint_state.name = joint_names
    current_joint_state.position = start_joint_angles

    moveit_robot_state = RobotState()
    moveit_robot_state.joint_state = current_joint_state
    move_group.set_start_state(moveit_robot_state)

    move_group.set_pose_target(destination_pose)
    print(current_joint_state)
    print(current_joint_state)
    print(move_group.plan())
    return planCompat(move_group.plan())


"""
    Creates a pick and place plan using the four states below.
    
    1. Pre Grasp - position gripper directly above target object
    2. Grasp - lower gripper so that fingers are on either side of object
    3. Pick Up - raise gripper back to the pre grasp position
    4. Place - move gripper to desired placement position
    Gripper behaviour is handled outside of this trajectory planning.
        - Gripper close occurs after 'grasp' position has been achieved
        - Gripper open occurs after 'place' position has been achieved
    https://github.com/ros-planning/moveit/blob/master/moveit_commander/src/moveit_commander/move_group.py
"""
def plan_pick_and_place(req):
    setup_scene()

    response = MoverServiceResponse()

    current_robot_joint_configuration = [
        math.radians(req.joints_input.joint_00),
        math.radians(req.joints_input.joint_01),
        math.radians(req.joints_input.joint_02),
        math.radians(req.joints_input.joint_03),
        math.radians(req.joints_input.joint_04),
        math.radians(req.joints_input.joint_05),
    ]

    # Pre grasp - position gripper directly above target object
    pre_grasp_pose = plan_trajectory(move_group, req.pick_pose, current_robot_joint_configuration)

    previous_ending_joint_angles = pre_grasp_pose.joint_trajectory.points[-1].positions
    response.trajectories.append(pre_grasp_pose)

    # Grasp - lower gripper so that fingers are on either side of object
    pick_pose = copy.deepcopy(req.pick_pose)
    pick_pose.position.z -= 0.075  # Static value coming from Unity, TODO: pass along with request
    grasp_pose = plan_trajectory(move_group, pick_pose, previous_ending_joint_angles)

    previous_ending_joint_angles = grasp_pose.joint_trajectory.points[-1].positions
    response.trajectories.append(grasp_pose)

    # Pick Up - raise gripper back to the pre grasp position
    pick_up_pose = plan_trajectory(move_group, req.pick_pose, previous_ending_joint_angles)

    previous_ending_joint_angles = pick_up_pose.joint_trajectory.points[-1].positions
    response.trajectories.append(pick_up_pose)

    # Place - move gripper to desired placement position
    place_pose = plan_trajectory(move_group, req.place_pose, previous_ending_joint_angles)

    previous_ending_joint_angles = place_pose.joint_trajectory.points[-1].positions
    response.trajectories.append(place_pose)

    # Post place - lift gripper after placement position
    place_pose = copy.deepcopy(req.place_pose)
    place_pose.position.z += 0.075 # Static value offset to lift up gripper
    post_place_pose = plan_trajectory(move_group, place_pose, previous_ending_joint_angles)

    previous_ending_joint_angles = post_place_pose.joint_trajectory.points[-1].positions
    response.trajectories.append(post_place_pose)

    move_group.clear_pose_targets()
    return response

def setup_scene():
    global move_group
    
    group_name = "arm"
    move_group = moveit_commander.MoveGroupCommander(group_name)

    # Add table collider to MoveIt scene
    scene = moveit_commander.PlanningSceneInterface()
    robot = moveit_commander.RobotCommander()
    rospy.sleep(2)

    p = PoseStamped()
    p.header.frame_id = move_group.get_planning_frame()
    p = set_pose(p, [0, 0, -0.02+0.77])
    scene.add_box("table", p, (1.2, 1.8, 0.01))

def set_pose(poseStamped, pose):
    '''
    pose is an array: [x, y, z]
    '''
    poseStamped.pose.position.x = pose[0]
    poseStamped.pose.position.y = pose[1]
    poseStamped.pose.position.z = pose[2]
    return poseStamped

In [10]:
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('jupyter_node_pub', anonymous=True)

ROSException: rospy.init_node() has already been called with different arguments: ('jupyter_node_sub', ['/home/jeyanth/.local/lib/python3.8/site-packages/ipykernel_launcher.py', '--ip=127.0.0.1', '--stdin=9273', '--control=9271', '--hb=9270', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"c8f5179c-21c9-47d9-845a-784d1a61fe9e"', '--shell=9272', '--transport="tcp"', '--iopub=9274', '--f=/tmp/tmp-17907DXUvZFAD0cIa.json'], True, None, False, False)

In [12]:
from geometry_msgs.msg import Point

setup_scene()
current_robot_joint_configuration = [math.radians(angle) for angle in robot_angles]
pick_pose = Pose()
pick_pose.position = Point(0.2,0.8,-0.3)
trajectories = []

# Pre grasp - position gripper directly above target object
pre_grasp_pose = plan_trajectory(move_group, pick_pose, current_robot_joint_configuration)

previous_ending_joint_angles = pre_grasp_pose.joint_trajectory.points[-1].positions
trajectories.append(pre_grasp_pose)

# Grasp - lower gripper so that fingers are on either side of object
new_pick_pose = copy.deepcopy(pick_pose)
new_pick_pose.position.z -= 0.075  # Static value coming from Unity, TODO: pass along with request
grasp_pose = plan_trajectory(move_group, new_pick_pose, previous_ending_joint_angles)

previous_ending_joint_angles = grasp_pose.joint_trajectory.points[-1].positions
trajectories.append(grasp_pose)
move_group.clear_pose_targets()

header: 
  seq: 0
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
name: 
  - shoulder_pan_joint
  - shoulder_lift_joint
  - elbow_joint
  - wrist_1_joint
  - wrist_2_joint
  - wrist_3_joint
position: [-0.853186465521407, -0.6887763487834198, -0.4374498419710069, -2.01425286202555, 1.5705830075915812, -2.4229964240275654]
velocity: []
effort: []
header: 
  seq: 0
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
name: 
  - shoulder_pan_joint
  - shoulder_lift_joint
  - elbow_joint
  - wrist_1_joint
  - wrist_2_joint
  - wrist_3_joint
position: [-0.853186465521407, -0.6887763487834198, -0.4374498419710069, -2.01425286202555, 1.5705830075915812, -2.4229964240275654]
velocity: []
effort: []
(False, joint_trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  joint_names: []
  points: []
multi_dof_joint_trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  joint_names: []
  poi

IndexError: list index out of range

In [ ]:
trajectories

In [ ]:
def post_color():
    pub = rospy.Publisher('UR_pub', UnityColor, queue_size=10)
    rate = rospy.Rate(10)  # 10hz

    while not rospy.is_shutdown():

        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        color = UnityColor(r, g, b, 1)
        pub.publish(color)
        rate.sleep()
        break

In [21]:
jr.publish('/UR_pub',URMoveitJoints)

# PyKDL

In [1]:
import kdl_parser_py.urdf as kdl_parser
import PyKDL as kdl
import rospkg

In [2]:
(status, tree) = kdl_parser.treeFromFile(rospkg.RosPack().get_path('ur_description')+'/urdf/ur3e.urdf')
print(tree.getChain)
chain = tree.getChain("base_link", "flange")

<bound method PyCapsule.getChain of base_link(q_nr: 0)
 	base_link_inertia(q_nr: 0)
 	shoulder_link(q_nr: 0)
 	upper_arm_link(q_nr: 1)
 	forearm_link(q_nr: 2)
 	wrist_1_link(q_nr: 3)
 	wrist_2_link(q_nr: 4)
 	wrist_3_link(q_nr: 5)
 	flange(q_nr: 0)
 	tool0(q_nr: 0)
 	
	
	
	
	
	
	
	
	
	base(q_nr: 0)
 	
	
>


In [4]:
fk_solver = kdl.ChainFkSolverPos_recursive(chain)
vel_ik_solver = kdl.ChainIkSolverVel_pinv(chain,0.0001,1000)
ik_solver = kdl.ChainIkSolverPos_NR(chain,fk_solver,vel_ik_solver,1000)

In [5]:
tcp_start = kdl.Frame()
Q_start = kdl.JntArray(6)
fk_solver.JntToCart(Q_start,tcp_start)
print(tcp_start)

[[ 2.51179e-26,          -1, 1.83697e-16;
            1, 6.27948e-26, 2.05103e-10;
 -2.05103e-10, 1.83697e-16,           1]
[     0.45675,     0.22315,      0.0665]]


In [6]:
tcp_goal = tcp_start*kdl.Frame(kdl.Rotation.RPY(0,0,0),kdl.Vector(-0.5,0,0))
Q_goal = kdl.JntArray(6)
ik_solver.CartToJnt(Q_start,tcp_goal,Q_goal)
Q_goal

[    -447.011    -352.514     -1204.7     840.929     746.794     358.142]

In [ ]:
kdl.ChainDynParam